In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install pyarrow
#!pip install fastparquet

In [4]:
PROJECT = "National Water Quality Assessment Program (NAWQA)"
START_DATE = "1991-01-01"  # verify start date
END_DATE = "2024-12-31"  # verify end date
CHARACTERISTIC = "Phosphorus"
FRACTION = "Dissolved"
# DESTINATION_BUCKET = os.environ.get('DESTINATION_BUCKET')

In [22]:
import pandas as pd

url = "s3://wma-uncertainty/nwqn-loadest-example/nwqn-samples.parquet/MonitoringLocationIdentifier=USGS-02087580"
samples = pd.read_parquet(url)
samples = samples[samples["USGSPCode"] == "631"]
samples["ResultMeasureValue"] = samples["ResultMeasureValue"].astype(float)

In [23]:
import pandas as pd

url = "s3://wma-uncertainty/nwqn-loadest-example/nwqn-streamflow.parquet/site_no=02087580"
daily = pd.read_parquet(url)
daily["00060_Mean"] = daily["00060_Mean"].astype(float)

In [25]:
from discontinuum.utils import aggregate_to_daily
from loadest_gp.providers.usgs import format_nwis_daily, format_wqp_samples

In [26]:
samples = format_wqp_samples(samples)
samples = aggregate_to_daily(samples)

/Users/thodson/Desktop/dev/software/discontinuum/src/loadest_gp/providers/usgs.py:255: UserWarning: Censored values have been removed from the dataset.
  warnings.warn(


In [27]:
daily = format_nwis_daily(daily)

In [28]:
daily

<xarray.Dataset> Size: 419kB
Dimensions:  (time: 26167)
Coordinates:
  * time     (time) datetime64[ns] 209kB 2002-03-01 2002-03-02 ... 2023-12-22
Data variables:
    flow     (time) float64 209kB 0.06824 0.7985 1.843 ... 8.467 1.235 0.2076

In [29]:
daily = daily.drop_duplicates("time")

In [30]:
import xarray as xr

training_data = xr.merge([samples, daily], join="inner")

In [31]:
training_data

<xarray.Dataset> Size: 7kB
Dimensions:        (time: 274)
Coordinates:
  * time           (time) datetime64[ns] 2kB 2002-03-19 ... 2023-08-04
Data variables:
    concentration  (time) float64 2kB 0.06 0.1 0.07 0.1 ... 0.06 0.12 0.12 0.13
    flow           (time) float64 2kB 0.2917 0.7476 0.08325 ... 0.2299 0.02549

In [41]:
training_data = training_data.dropna("time", how="any")

In [42]:
%%time
# select an engine
# from loadest_gp import LoadestGPMarginalPyMC as LoadestGP
from loadest_gp import LoadestGPMarginalGPyTorch as LoadestGP

model = LoadestGP()

model.fit(target=training_data["concentration"], covariates=training_data[["time", "flow"]])

100%|███████████████████| 100/100 [00:00<00:00, 186.98it/s, loss=1.36]

CPU times: user 2.66 s, sys: 449 ms, total: 3.11 s
Wall time: 555 ms


In [12]:
samples

<xarray.Dataset> Size: 6kB
Dimensions:        (time: 358)
Coordinates:
  * time           (time) datetime64[ns] 3kB 2001-10-12 ... 2023-08-04
Data variables:
    concentration  (time) float64 3kB 0.05 0.11 0.08 0.08 ... 0.08 0.15 0.13

In [32]:
samples["ResultMeasureValue"].count()

192

In [34]:
len(samples["ResultMeasureValue"])

992

In [5]:
# from loadest_gp.providers.usgs import format_wqp_samples

ModuleNotFoundError: No module named 'gpytorch'

In [1]:
import pandas as pd

samples_url = "s3://wma-uncertainty/nwqn-loadest-example/nwqn-samples.parquet"
site = "01054200"
df = pd.read_parquet(f"{samples_url}/MonitoringLocationIdentifier=USGS-{site}")

OSError: When getting information for key 'nwqn-loadest-example/nwqn-samples.parquet/MonitoringLocationIdentifier=USGS-01054200' in bucket 'wma-uncertainty': AWS Error UNKNOWN (HTTP status 400) during HeadObject operation: No response body.

In [3]:
from loadest_gp.providers import usgs

In [4]:
import pandas as pd

site_df = pd.read_csv(
    "../examples/nawqa-loadest-example/NWQN_sites.csv",
    comment="#",
    dtype={"SITE_QW_ID": str, "SITE_FLOW_ID": str},
)

In [5]:
site_list = site_df["SITE_QW_ID"].to_list()

In [6]:
site_list[0]

'01054200'

In [76]:
samples = usgs.get_samples(
    site=f"USGS-{site_list[0]}",
    start_date=START_DATE,
    end_date=END_DATE,
    characteristic="Inorganic nitrogen (nitrate and nitrite)",
    provider="NWIS",
    fraction="Dissolved",
    project=PROJECT,
    # filter_pcodes=[665],
)

/Users/thodson/micromamba/envs/discontinuum-dev/lib/python3.11/site-packages/dataretrieval/wqp.py:494: UserWarning: USGS discrete water quality data availability and format are changing. Beginning in March 2024 the data obtained from legacy profiles will not include new USGS data or recent updates to existing data. To view the status of changes in data availability and code functionality, visit: https://doi-usgs.github.io/dataRetrieval/articles/Status.html. If you have additional questions about these changes, email CompTools@usgs.gov.
  warnings.warn(('USGS discrete water quality data availability '
/Users/thodson/Desktop/dev/software/discontinuum/src/loadest_gp/providers/usgs.py:276: UserWarning: Censored values have been removed from the dataset.
  warnings.warn(


In [72]:
samples

<xarray.Dataset> Size: 192B
Dimensions:        (time: 12)
Coordinates:
  * time           (time) datetime64[ns] 96B 2019-10-08T09:30:00 ... 2022-02-...
Data variables:
    concentration  (time) float64 96B 0.02 0.01 0.02 0.01 ... 0.02 0.02 0.03
Attributes:
    id:         01054200
    name:       Wild River at Gilead, Maine
    latitude:   44.39055556
    longitude:  -70.9797222

In [11]:
from dataretrieval import wqp

In [73]:
df, meta = wqp.get_results(
    siteid=f"USGS-{site_list[0]}",
    provider="NWIS",
    startDateLo="01-01-1990",
    startDateHi="12-31-2024",
    characteristicName="Inorganic nitrogen (nitrate and nitrite)",
    project=PROJECT,
    # dataProfile='narrowResult',
)

/Users/thodson/micromamba/envs/discontinuum-dev/lib/python3.11/site-packages/dataretrieval/wqp.py:494: UserWarning: USGS discrete water quality data availability and format are changing. Beginning in March 2024 the data obtained from legacy profiles will not include new USGS data or recent updates to existing data. To view the status of changes in data availability and code functionality, visit: https://doi-usgs.github.io/dataRetrieval/articles/Status.html. If you have additional questions about these changes, email CompTools@usgs.gov.
  warnings.warn(('USGS discrete water quality data availability '


In [75]:
df.shape

(62, 63)

In [42]:
df = df[df["USGSPCode"] == 631]

In [57]:
df["ResultSampleFractionText"].unique()

array(['Dissolved'], dtype=object)

In [58]:
df["USGSPCode"].unique()

array([631])

In [46]:
df["CharacteristicName"]

55       Inorganic nitrogen (nitrate and nitrite)
111      Inorganic nitrogen (nitrate and nitrite)
171      Inorganic nitrogen (nitrate and nitrite)
226      Inorganic nitrogen (nitrate and nitrite)
283      Inorganic nitrogen (nitrate and nitrite)
                           ...                   
12557    Inorganic nitrogen (nitrate and nitrite)
12594    Inorganic nitrogen (nitrate and nitrite)
12631    Inorganic nitrogen (nitrate and nitrite)
12698    Inorganic nitrogen (nitrate and nitrite)
12798    Inorganic nitrogen (nitrate and nitrite)
Name: CharacteristicName, Length: 242, dtype: object

In [47]:
df, _ = nwis.get_pmcodes("99891")

In [49]:
df["parm_nm"]

0    Phosphorus, water, unfiltered, modified jirka ...
Name: parm_nm, dtype: object

In [17]:
import dataretrieval.wqp as wqp
from harmonize_wq import wrangle

# File for area of interest
aoi_url = r"https://raw.githubusercontent.com/USEPA/harmonize-wq/main/harmonize_wq/tests/data/PPBays_NCCA.geojson"

# Build query
query = {
    "characteristicName": [
        "Temperature, water",
        "Depth, Secchi disk depth",
    ]
}
query["bBox"] = wrangle.get_bounding_box(aoi_url)
query["dataProfile"] = "narrowResult"

# Run query
res_narrow, md_narrow = wqp.get_results(**query)

# dataframe of downloaded results
res_narrow

/Users/thodson/micromamba/envs/discontinuum-dev/lib/python3.11/site-packages/dataretrieval/wqp.py:494: UserWarning: USGS discrete water quality data availability and format are changing. Beginning in March 2024 the data obtained from legacy profiles will not include new USGS data or recent updates to existing data. To view the status of changes in data availability and code functionality, visit: https://doi-usgs.github.io/dataRetrieval/articles/Status.html. If you have additional questions about these changes, email CompTools@usgs.gov.
  warnings.warn(('USGS discrete water quality data availability '


KeyboardInterrupt: 

In [2]:
from dataretrieval import nwis

nwis.get_dv(sites="03339000", parameterCd="00060")

(                            site_no  00060_Mean 00060_Mean_cd
 datetime                                                     
 2024-08-14 00:00:00+00:00  03339000       103.0             P,
 NWIS_Metadata(url=https://waterservices.usgs.gov/nwis/dv?format=json&parameterCd=00060&sites=03339000))